# Rekurencyjne Sieci Neuronowe (RNN)

### Importy i Utilsy  (odpalić i schować )

In [ ]:
# imports 
import torch
import os
import unicodedata
import string
import numpy as np
from typing import Tuple, Optional, List

from torch.nn.functional import cross_entropy

import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score

from torch.utils.data import Dataset, DataLoader

all_letters = string.ascii_letters
n_letters = len(all_letters)


class ListDataset(Dataset):
    
    def __init__(self, data, targets):
        
        self.data = data
        self.targets = targets
        
    def __getitem__(self, ind):
        
        return self.data[ind], self.targets[ind]
    
    def __len__(self):
        return len(self.targets)

    
def unicode_to__ascii(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
                                                                 and c in all_letters)
                   

def read_lines(filename: str) -> List[str]:
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to__ascii(line) for line in lines]


def letter_to_index(letter: str) -> int:
    return all_letters.find(letter)


def line_to_tensor(line: str) -> torch.Tensor:
    tensor = torch.zeros(len(line), n_letters)
    for i, letter in enumerate(line):
        tensor[i][letter_to_index(letter)] = 1
    return tensor

## Dane sekwencyjne

Modele, którymi zajmowaliśmy się wcześniej zakładały konkretny kształt danych. Dla przykładu klasyczna sieć neuronowa fully-connected zakładała, że na wejściu dostanie wektory rozmiaru 784 - dla wektorów o innej wymiarowości i innych obiektów model zwyczajnie nie będzie działać.

Takie założenie bywa szczególnie niewygodne przy pracy z niektórymi typami danych, takimi jak:
* językiem naturalny (słowa czy zdania mają zadanej z góry liczby znaków)
* szeregi czasowe (dane giełdowe ciągną się właściwie w nieskończoność) 
* dźwięk (nagrania mogą być krótsze lub dłuższe).

Do rozwiązania tego problemu służą rekuencyjne sieci neuronowe (*recurrent neural networks, RNNs*), które zapamiętują swój stan z poprzedniej iteracji.

### Ładowanie danych
Poniższe dwie komórki ściągają dataset nazwisk z 18 różnych narodowości. Każda litera w danym nazwisku jest zamieniana na jej indeks z alfabetu w postaco kodowania "one-hot". Podsumowując, każde nazwisko jest binarną macierzą rozmiaru `n_letters` $\times$ `len(name)`. 

Dodatkowo ponieważ ten dataset jest mocno niezbalansowany uzyjemy specjalnego samplera do losowania przykładów treningowych tak aby do uczenia sieć widziała tyle samo przykładów z każdej klasy.

Ponieważ nazwiska mogą mieć różne długości będziemy rozważać `batch_size = 1` w tym notebooku (choć implementacje modeli powinny działać dla dowolnych wartości `batch_size`!)

In [ ]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zipq

--2021-01-11 13:31:24--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.32.204.34, 13.32.204.93, 13.32.204.49, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.32.204.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip.2’

data.zip.2          100%[===================>]   2.75M  --.-KB/s    in 0.04s   

2021-01-11 13:31:24 (64.3 MB/s) - ‘data.zip.2’ saved [2882130/2882130]

unzip:  cannot find or open data.zipq, data.zipq.zip or data.zipq.ZIP.


In [ ]:
# NOTE: you can change the seed or remove it completely if you like
torch.manual_seed(1337)

data_dir = 'data/names'

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

data = []
targets = [] 
label_to_idx = {}

# read each natonality file and process data 
for label, file_name in enumerate(os.listdir(data_dir)):
    
    label_to_idx[label] = file_name.split('.')[0].lower()
    
    names = read_lines(os.path.join(data_dir, file_name))
    data += [line_to_tensor(name) for name in names]
    targets += len(names) * [label]

# split into train and test indices
test_frac = 0.1
n_test = int(test_frac * len(targets))
test_ind = np.random.choice(len(targets), size=n_test, replace=False)
train_ind = np.setdiff1d(np.arange(len(targets)), test_ind)

targets = torch.tensor(targets)
train_targets = targets[train_ind]

# calculate weights for BalancedSampler
uni, counts = np.unique(train_targets, return_counts=True)
weight_per_class = len(targets) / counts
weight = [weight_per_class[c] for c in train_targets]
# preapre the sampler
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weight, num_samples=len(weight)) 

train_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in train_ind], targets=train_targets)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1, sampler=sampler)

test_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in test_ind], targets=targets[test_ind])
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

In [ ]:
# check out the content of the dataset
for i, (x, y) in enumerate(train_loader):
    break

print("x.shape:", x.shape)
print("name: ", end="")
for letter_onehot in x[0]:
    print(all_letters[torch.argmax(letter_onehot)], end="")

print("\ny:", label_to_idx[y.item()])

x.shape: torch.Size([1, 5, 52])
name: Abadi
y: arabic


## Zadanie 1. (2 pkt.)

Zaimplementuj "zwykłą" sieć rekurencyjną. 
![rnn](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

* W klasie `RNN` należy zainicjalizować potrzebne wagi oraz zaimplementować główną logikę dla pojedynczej chwili czasowej $x_t$
* Wyjście z sieci możemy mieć dowolny rozmiar, potrzebna jest również warstwa przekształacjąca stan ukryty na wyjście.
* W pętli uczenia należy dodać odpowiednie wywołanie sieci. HINT: pamiętać o iterowaniu po wymiarze "czasowym".


In [ ]:
class RNN(torch.nn.Module):
    
    def __init__(self, 
                 input_size: int,
                 hidden_size: int, 
                 output_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        :param output_size: int
            Desired dimensionality of the output vector
        """
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.input_to_hidden = torch.nn.Linear(in_features=input_size + hidden_size, 
                                         out_features=hidden_size)  
        
        self.hidden_to_output = torch.nn.Linear(in_features=hidden_size, out_features=output_size)
    
    # for the sake of simplicity a single forward will process only a single timestamp 
    def forward(self, 
                input: torch.tensor, 
                hidden: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        :param hidden: torch.tensor
            Representation of the memory of the RNN from previous timestep
            shape [batch_size, hidden_size]
        """
        
        combined = torch.cat([input, hidden], dim=1) 
        hidden = torch.tanh(self.input_to_hidden(combined)) # ???
        output =  self.hidden_to_output(hidden) # ???
        return output, hidden
    
    def init_hidden(self, batch_size: int) -> torch.Tensor:
        """
        Returns initial value for the hidden state
        """
        return torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda()

### Pętla uczenia

In [ ]:
n_class = len(label_to_idx)

# initialize network and optimizer
rnn = RNN(n_letters, 256, n_class).cuda()
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.01)   

# we will train for only a single epoch 
epochs = 1


# main loop
for epoch in range(epochs):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):  
        
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        # get initial hidden state
        hidden = rnn.init_hidden(x.shape[0])
        
        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the 2nd, time dimensiotn

        seq_len = x.shape[1]
        
        for t in range(seq_len): 
            x_t = x[:, t]
            output, hidden = rnn(input=x_t, hidden=hidden)
            
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()  
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []
    

# evaluate on the test set
with torch.no_grad():
    ps = []
    ys = []
    correct = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.cuda()
        ys.append(y.numpy())

        hidden = rnn.init_hidden(x.shape[0])
        seq_len = x.shape[1]
        for t in range(seq_len): 
            x_t = x[:, t]
            output, hidden = rnn(input=x_t, hidden=hidden)

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.15, "You should get over 0.15 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 2.935
Epoch: 0 Progress:  6% Loss: 2.860
Epoch: 0 Progress: 11% Loss: 2.766
Epoch: 0 Progress: 17% Loss: 2.495
Epoch: 0 Progress: 22% Loss: 2.258
Epoch: 0 Progress: 28% Loss: 2.135
Epoch: 0 Progress: 33% Loss: 2.013
Epoch: 0 Progress: 39% Loss: 1.916
Epoch: 0 Progress: 44% Loss: 1.865
Epoch: 0 Progress: 50% Loss: 1.783
Epoch: 0 Progress: 55% Loss: 1.847
Epoch: 0 Progress: 61% Loss: 1.778
Epoch: 0 Progress: 66% Loss: 1.673
Epoch: 0 Progress: 72% Loss: 1.694
Epoch: 0 Progress: 77% Loss: 1.672
Epoch: 0 Progress: 83% Loss: 1.607
Epoch: 0 Progress: 89% Loss: 1.611
Epoch: 0 Progress: 94% Loss: 1.623
Epoch: 0 Progress: 100% Loss: 1.673
Final F1 score: 0.17


## Zadanie 2. (0.5 pkt.)
Zaimplementuj funkcje `predict`, która przyjmuje nazwisko w postaci stringa oraz model RNN i wypisuje 3 najlepsze predykcje narodowości dla tego nazwiska razem z ich logitami.

**Hint**: Przyda się tutaj jedna z funkcji z pierwszej komórki notebooka.

In [ ]:
def predict(name: str, rnn: RNN):
    x = line_to_tensor(name).unsqueeze(0).cuda()

    hidden = rnn.init_hidden(x.shape[0])
    seq_len = x.shape[1]
    for t in range(seq_len): 
        x_t = x[:, t]
        output, hidden = rnn(input=x_t, hidden=hidden)

    res = output.topk(3, dim=1)
 
    for score, ind in zip(res.values[0], res.indices[0]):
        print(f"\t{label_to_idx[ind.item()]}: {score:.2f}")

In [ ]:
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]

for name in some_names:
    print(name)
    predict(name, rnn)

Satoshi
	japanese: 3.69
	italian: 3.40
	arabic: 2.52
Jackson
	scottish: 4.39
	russian: 2.74
	english: 2.72
Schmidhuber
	german: 3.60
	czech: 2.94
	dutch: 2.84
Hinton
	scottish: 2.68
	english: 2.60
	irish: 1.67
Kowalski
	polish: 6.46
	czech: 3.91
	russian: 3.10


## Zadanie 3 (4 pkt.)
Ostatnim zadaniem jest implementacji komórki i sieci LSTM. 

![lstm](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

* W klasie `LSTMCell` ma znaleźć się główna loginka LSTMa, czyli wszystkie wagi do stanów `hidden` i `cell` jak i bramek kontrolujących te stany. 
* W klasie `LSTM` powinno znaleźć się wywołanie komórki LSTM, HINT: poprzednio było w pętli uczenia, teraz przenisiemy to do klasy modelu.
* W pętli uczenia należy uzupełnić brakujące wywołania do uczenia i ewaluacji modelu.

Zdecydowanie polecam [materiały Chrisa Olaha](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) do zarówno zrozumienia jak i ściągi do wzorów.

Zadaniem jest osiągnięcie wartości $f1_score$ lepszej niż na sieci RNN, przy prawidłowej implementacji nie powinno być z tym problemów używając podanych hiperparametrów. Dozwolona jest oczywiście zmiana `random seed`.

#### Komórka LSTM

In [ ]:
class LSTMCell(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTMCell, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        # initialize LSTM weights 
        # NOTE: there are different approaches that are all correct 
        # (e.g. single matrix for all input opperations), you can pick
        # whichever you like for this task
    
        self.W_f = torch.nn.Linear(in_features=hidden_size + input_size, out_features=hidden_size)
        self.W_i = torch.nn.Linear(in_features=hidden_size + input_size, out_features=hidden_size)
        self.W_c = torch.nn.Linear(in_features=hidden_size + input_size, out_features=hidden_size)
        self.W_o = torch.nn.Linear(in_features=hidden_size + input_size, out_features=hidden_size)

    def forward(self, 
                input: torch.tensor, 
                states: Tuple[torch.tensor, torch.tensor]) -> Tuple[torch.tensor, torch.tensor]:
        
        hidden, cell = states
        
        # Compute input, forget, and output gates
        # then compute new cell state and hidden state
        # see http://colah.github.io/posts/2015-08-Understanding-LSTMs/ 
        
        combined = torch.cat([input, hidden], dim=1)
        
        forget_gate = torch.sigmoid(self.W_f(combined))
        input_gate = torch.sigmoid(self.W_i(combined))
        
        cell_update = torch.sigmoid(self.W_c(combined))
        cell = forget_gate * cell + input_gate * cell_update
        
        output = torch.sigmoid(self.W_o(combined))
        hidden = output * torch.tanh(cell)
        
        return hidden, cell

### Klasa modelu LSTM

In [ ]:
class LSTM(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = LSTMCell(input_size=input_size, hidden_size=hidden_size)
        
    def forward(self, 
                input: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        Returns Tuple of two torch.tensors, both of shape [seq_len, batch_size, hidden_size]
        """
        
        batch_size = input.shape[0]
        
        hidden, cell = self.init_hidden_cell(batch_size)
        
        hiddens = []
        cells = []
        
        # this time we will process the whole sequence in the forward method
        # as oppose to the previous exercise, remember to loop over the timesteps
        
        time_steps = input.shape[1]
        
        for t in range(time_steps):
            x = input[:, t]
            hidden, cell = self.cell(x, (hidden, cell))
            
            hiddens.append(hidden)
            cells.append(cell)

        return torch.stack(hiddens), torch.stack(cells)
    
    def init_hidden_cell(self, batch_size):
        """
        Returns initial value for the hidden and cell states
        """
        return (torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda(), 
                torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda())

### Pętla uczenia

In [ ]:
from itertools import chain

# torch.manual_seed(1337)

# build data loaders
train_loader = DataLoader(train_dataset, batch_size=1, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=1)

# initialize the lstm with an additional cliassifier layer at the top
lstm = LSTM(input_size=len(all_letters), hidden_size=128).cuda()
clf = torch.nn.Linear(in_features=128, out_features=len(label_to_idx)).cuda()

# initialize a optimizer
params = chain(lstm.parameters(), clf.parameters())
optimizer = torch.optim.Adam(params, lr=0.01) 

# we will train for only a single epoch 
epoch = 1

# main loop
for epoch in range(epoch):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):   
        
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the sequence length here
        
        hidden, state = lstm(x)
        output = clf(hidden[-1])
        # calucate the loss
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()                                
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []

# evaluate on the test set
with torch.no_grad():
    
    ps = []
    ys = []
    for i, (x, y) in enumerate(test_loader): 
        
        x = x.cuda()
        ys.append(y.numpy())
        
        hidden, state = lstm(x)
        output = clf(hidden[-1])

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.18, "You should get over 0.18 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 1.986
Epoch: 0 Progress:  6% Loss: 2.691
Epoch: 0 Progress: 11% Loss: 2.015
Epoch: 0 Progress: 17% Loss: 1.836
Epoch: 0 Progress: 22% Loss: 1.636
Epoch: 0 Progress: 28% Loss: 1.516
Epoch: 0 Progress: 33% Loss: 1.389
Epoch: 0 Progress: 39% Loss: 1.345
Epoch: 0 Progress: 44% Loss: 1.240
Epoch: 0 Progress: 50% Loss: 1.250
Epoch: 0 Progress: 55% Loss: 1.193
Epoch: 0 Progress: 61% Loss: 1.104
Epoch: 0 Progress: 66% Loss: 1.040
Epoch: 0 Progress: 72% Loss: 1.122
Epoch: 0 Progress: 77% Loss: 0.911
Epoch: 0 Progress: 83% Loss: 1.007
Epoch: 0 Progress: 89% Loss: 0.957
Epoch: 0 Progress: 94% Loss: 0.997
Epoch: 0 Progress: 100% Loss: 0.910
Final F1 score: 0.21


## Zadanie 4. (0.5 pkt.)
Zaimplementuj analogiczną do funkcji `predict` z zadania 2 dla modelu `lstm+clf`.


In [ ]:
def predict_lstm(name: str, lstm: LSTM, clf: torch.nn.Module):
    x = line_to_tensor(name).unsqueeze(0).cuda()

    hidden, state = lstm(input=x)
    output = clf(hidden[-1])

    res = output.topk(3, dim=1)
 
    for score, ind in zip(res.values[0], res.indices[0]):
        print(f"\t{label_to_idx[ind.item()]}: {score:.2f}")

In [ ]:
# test your lstm predictor
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]
    
for name in some_names:
    print(name)
    predict_lstm(name, lstm, clf)

Satoshi
	arabic: 0.03
	japanese: -1.19
	russian: -3.46
Jackson
	scottish: 0.09
	english: -1.60
	russian: -3.67
Schmidhuber
	german: 0.49
	russian: -1.44
	english: -2.81
Hinton
	french: -0.04
	english: -0.21
	irish: -0.96
Kowalski
	polish: 2.81
	czech: -4.55
	russian: -4.67
